In [1]:
import os
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

In [2]:
# Imports
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import ViTModel
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

E0000 00:00:1752384950.683747    3917 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752384950.753253    3917 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752384951.340300    3917 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752384951.340342    3917 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752384951.340345    3917 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752384951.340347    3917 computation_placer.cc:177] computation placer already registered. Please check linka

In [3]:
# Configs
MODEL_NAME = "google/vit-base-patch16-224"
FER_CSV_PATH = "../emotions_csv/fer2013.csv"
MODEL_SAVE_PATH = "saved_model/vit_emotion_head.pth"
EMOTION_CLASSES = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

BATCH_SIZE = 64
EPOCHS = 30
LR = 1e-4
PATIENCE = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Custom Dataset
class FERDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        pixels = np.array(row['pixels'].split(), dtype='float32').reshape(48, 48)
        img = Image.fromarray(pixels).convert("RGB").resize((224, 224))
        if self.transform:
            img = self.transform(img)
        label = int(row['emotion'])
        return img, label

In [5]:
# Data Load and augmentation
df = pd.read_csv(FER_CSV_PATH)
train_df = df[df['Usage'] == 'Training'].reset_index(drop=True)
val_df = df[df['Usage'] == 'PublicTest'].reset_index(drop=True)

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = FERDataset(train_df, transform)
val_dataset = FERDataset(val_df, transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
# Model With Emotion Head Only
class EmotionClassifier(nn.Module):
    def __init__(self, vit, num_classes=7, dropout=0.3):
        super().__init__()
        self.vit = vit
        self.dropout = nn.Dropout(dropout)
        self.emotion_head = nn.Sequential(
            nn.Linear(vit.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.vit(pixel_values=x).last_hidden_state[:, 0]
        x = self.dropout(x)
        return self.emotion_head(x)

In [7]:
# Load pre-trained ViT
vit = ViTModel.from_pretrained(MODEL_NAME)

# Freeze all layers
for param in vit.parameters():
    param.requires_grad = False

# Unfreeze last 2 transformer blocks
for name, param in vit.named_parameters():
    if "encoder.layer.10" in name or "encoder.layer.11" in name:
        param.requires_grad = True

# Initialize model, loss, optimizer
model = EmotionClassifier(vit).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Training loop with early stopping
best_val_acc = 0
patience_counter = 0
train_losses, val_accuracies = [], []

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    train_loss = total_loss / len(train_loader)
    acc = 100 * correct / len(train_dataset)
    train_losses.append(train_loss)

    print(f"Train Loss: {train_loss:.4f}, Accuracy: {acc:.2f}%")

    # Validation
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    val_acc = 100 * val_correct / len(val_dataset)
    val_accuracies.append(val_acc)
    print(f"Validation Accuracy: {val_acc:.2f}%")

    # Early stopping check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"Saved improved model at Epoch {epoch+1}")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping triggered.")
            break

Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████| 449/449 [08:42<00:00,  1.16s/it]


Train Loss: 1.3498, Accuracy: 55.12%
Validation Accuracy: 60.41%
Saved improved model at Epoch 1


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████| 449/449 [08:02<00:00,  1.07s/it]


Train Loss: 1.2016, Accuracy: 62.91%
Validation Accuracy: 63.58%
Saved improved model at Epoch 2


Epoch 3/30: 100%|█████████████████████████████████████████████████████████████████████| 449/449 [07:56<00:00,  1.06s/it]


...

Train Loss: 0.8033, Accuracy: 84.06%
Validation Accuracy: 66.17%


Epoch 10/30: 100%|████████████████████████████████████████████████████████████████████| 449/449 [08:15<00:00,  1.10s/it]


Train Loss: 0.7623, Accuracy: 85.91%
Validation Accuracy: 67.43%
Saved improved model at Epoch 10


Epoch 11/30:   9%|██████▎                                                              | 41/449 [00:46<07:39,  1.13s/it]


KeyboardInterrupt: 

In [ ]:
# Plot
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.title('Training Loss and Validation Accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Load best model
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

In [ ]:
# Evaluate
all_preds, all_labels = [], []
model.eval()
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=EMOTION_CLASSES))

In [ ]:
# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=EMOTION_CLASSES)
disp.plot(xticks_rotation=45, cmap='Blues')
plt.title("Confusion Matrix")
plt.grid(False)
plt.show()

In [ ]:
# Show predictions
def show_predictions(model, dataset, num=6):
    model.eval()
    fig, axes = plt.subplots(1, num, figsize=(15, 3))
    for i in range(num):
        img, label = dataset[i]
        with torch.no_grad():
            pred = model(img.unsqueeze(0).to(device)).argmax(1).item()
        axes[i].imshow(img.permute(1, 2, 0).numpy() * 0.5 + 0.5)
        axes[i].set_title(f"GT: {EMOTION_CLASSES[label]}\nPred: {EMOTION_CLASSES[pred]}")
        axes[i].axis("off")
    plt.tight_layout()
    plt.show()

show_predictions(model, val_dataset)

print(f"Training complete. Best model saved to {MODEL_SAVE_PATH}")